# Fruit and Vegetable Prices


## Data set selection

> In this section, you will need to provide the following information about the selected data set:

### Source (with link) 
The dataset comes from the USDA Economic Research Service – Fruit and Vegetable Prices Database.  
Link: https://www.ers.usda.gov/data-products/fruit-and-vegetable-prices/

---

### Fields

- **Fruit** – Name of the fruit  
- **Form** – The form in which the fruit is sold (Fresh, Canned, Juice, Frozen, etc.)  
- **RetailPrice** – Price per unit as sold (e.g., per pound, per pint)  
- **RetailPriceUnit** – Unit of pricing  
- **Yield** – Edible yield percentage after removing waste (peel, core, etc.)  
- **CupEquivalentSize** – How much product is needed to make 1 edible cup  
- **CupEquivalentUnit** – Unit used for the cup conversion  
- **CupEquivalentPrice** – Cost to obtain one edible cup of the fruit  

---

### License
Creative Commons CCZero  
Link: http://www.opendefinition.org/licenses/cc-zero

### Data set selection rationale

I selected this dataset because this dataset could provide real-world economic and nutrition data that are useful for health insights. It includes multiple variables that I can use for statistical analysis. The dataset is clean overall, I didn't have to preprocess it. By analyzing this dataset, I might be able to create analysis related to fruit affordability, and which fruits bring most value based on its price and yield.

### Questions to be answered

> Using statistical analysis and visualization, what questions would you like to be able answer about this dataset.
> This could include questions such as:

 - What is the price distribution for different fruits?  
 We can compare how fruit prices varies in cost
 - How does the form of the fruit affect the retail price?  
 This helps determine whether processing tends to make specific fruit more or less expensive
 - What is the relationship between retail price and cup equivalent price?  
 This helps understand how much edible fruit consumers actually get for the price they pay
 - How does yield impact the cost of edible fruit?
 This may answer if a fruit with low yield may be cheap per pound but be more expensive per serving
 - What are top 5 affordable fruits based on cup equivalent price?  
 This helps identify the most 5 affordable fruits. With the data collected, this would answer what fruits are the best for budget planning  
 - What is the distribution of fruit forms?
 This would answer which type of fruit products dominate the dataset. For stakeholders, it would help them understand what products consumers have access to.

### Visualization ideas

> Provide a few examples of what you plan to visualize to answer the questions you posed in the previous section. In this project, you will be producing 6-8 visualizations. You will also be producing an interactive chart using Plotly.
1. Histogram for the distribution of retail prices. This would show how fruit price vary across the dataset.  
2. Bar chart for average retail price by fruit. This would allow comparison of which fruits are more or less expensive.  
3. Grouped bar chart for retail price vs form. This would show whether fresh, canned, frozen, or joice tend to be cheaper or more expensive.  
4. Scatter plot for retail price vs cup equivalent price. This would help identify correlation between retail price and cup equivalent price so that stakeholders can what fruits bring the good value.  
5. Scatter plot for the impact of yield on cup equivalent price. This would show how edible yield affects how much the consumers pay. Low-yield fruit may seem cheap per pound but expensive per cup.  
6. Pie chart for top 5 most affordable fruits. This would help identify the cheapest fruit based on cup equivalent price. This would be used for budget planning.  
7. Pie chart for distribution of fruit forms. This would help understand which types of fruit products dominate the dataset.



In [1]:
# 🚀 Importing some libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Idea 1
>
